In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install -q transformers datasets sentencepiece

!pip uninstall -y -q transformers accelerate
!pip install -q transformers accelerate

!pip install -q rouge-score

!pip install -q torchmetrics

!pip install -q git+https://github.com/huggingface/accelerate

!git clone -q https://github.com/huggingface/transformers.git
!cd transformers
!pip install  -q -e.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 30.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 9.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [3]:

# model_name_or_path = 'google/mt5-small'
model_name_or_path = '/content/drive/MyDrive/mT5_QA/checkpoint-13234'
from transformers import AutoTokenizer, MT5ForConditionalGeneration
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = MT5ForConditionalGeneration.from_pretrained(model_name_or_path)

comment the following lines if you want to fine-tune on different datasets

In [ ]:
# from datasets import concatenate_datasets, load_dataset

# train_data = load_dataset("Gholamreza/pquad", split="train")
# valid_data = load_dataset("Gholamreza/pquad", split="validation")
# test_data = load_dataset("Gholamreza/pquad", split="test")

# concat_dataset = concatenate_datasets([train_data, valid_data, test_data])
# dataset=concat_dataset

In [5]:
from datasets import Dataset
import random
import json

# data_files = ['total_augmented.json', 'train.json', 'dev.json']

data_files = ['train_v5.json', 'back_translated_v5.json', 'extra_translations_v5.json', 'dev_v5.json']

total_data_list = []

for file in data_files:
  path = '/content/drive/MyDrive/' + file
  with open(path, 'r', encoding = 'utf-8-sig') as f:
    content = json.load(f)
    total_data_list = total_data_list + content

random.shuffle(total_data_list)

dataset = Dataset.from_list(total_data_list)

In [6]:
dataset

Dataset({
    features: ['question', 'pq_id', 'context', 'answers'],
    num_rows: 19596
})

In [7]:
# del concat_dataset, train_data, valid_data

small_dataset = dataset

# small_dataset= dataset.filter(lambda example: len(example['answers']['text'])>0)
# small_dataset= small_dataset.filter(lambda example: 'چرا' not in example['context'])
split_data = small_dataset.train_test_split(test_size=0.1, seed= 42, shuffle=True)

dataset = split_data

del split_data

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'pq_id', 'context', 'answers'],
        num_rows: 17636
    })
    test: Dataset({
        features: ['question', 'pq_id', 'context', 'answers'],
        num_rows: 1960
    })
})

In [9]:
# def format_dataset(example):
#   return {'input': 'question: ' + example['question'] + ' context: ' + example['context'], 'target': example['answers'][0]['text']}

def format_dataset(example):
  context = example['context']
  question = example['question']
  # answer = example['answers']['text'][0]
  answer = example['answers'][0]['text']
  question_plus = f"question: {str(question)}"
  question_plus += f" context: {str(context)} </s>"

  answer_plus = answer
  answer_plus = f"{answer_plus} </s>"

  # return {'input': 'question: ' + example['question'] + ' context: ' + example['context'], 'target': example['answers'][0]['text']}
  return {'input': question_plus, 'target': answer_plus}

dataset = dataset.map(format_dataset, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/17636 [00:00<?, ? examples/s]

Map:   0%|          | 0/1960 [00:00<?, ? examples/s]

In [10]:
test_data = dataset['test']

test_data[0]

# train_ds=  dataset['train'].map(encode)
# valid_ds=  dataset['test'].map(encode)


{'input': 'question: سخنان حضرت ایوب (ع) برای چه کسانی یادآوری است؟ context: و خانواده\u200cاش را به او بخشیدیم، و همانند آنها را بر آنان افزودیم، تا رحمتی از سوی ما باشد و تذکّری برای اندیشمندان. </s>',
 'target': 'اندیشمندان </s>'}

In [11]:
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input'], truncation=True, padding='max_length', max_length=564)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target'], truncation=True, padding='max_length', max_length=64)

    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids'],
        'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings

In [12]:
dataset = dataset.map(convert_to_features, batched=True, remove_columns=dataset['train'].column_names)

columns = ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask']

# train_ds.set_format(type='torch', columns=columns)
# valid_ds.set_format(type='torch', columns=columns)

dataset.set_format(type='torch', columns=columns)

Map:   0%|          | 0/17636 [00:00<?, ? examples/s]

Map:   0%|          | 0/1960 [00:00<?, ? examples/s]

In [13]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,Seq2SeqTrainingArguments, Seq2SeqTrainer
# data_collator = default_data_collator
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [14]:
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments

# set training arguments - Feel free to adapt it
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/mT5_QA",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    do_train=True,
    do_eval=True,
    save_strategy="epoch"
    # save_strategy="no",
    #eval_steps=1000,
    # push_to_hub=False
    #fp16=True,
)


In [15]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    # data_collator=T2TDataCollator(),
    data_collator=data_collator,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics
)

In [16]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [17]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.036500,0.020477


TrainOutput(global_step=4409, training_loss=0.045607982132812835, metrics={'train_runtime': 1641.4233, 'train_samples_per_second': 10.744, 'train_steps_per_second': 2.686, 'total_flos': 1.027210286186496e+16, 'train_loss': 0.045607982132812835, 'epoch': 1.0})

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# prompt = "question: خداوند در طرف راست کدام کوه با بنی اسرائیل وعده گذارد؟ context: اي بني اسرائيل ما شما را از (چنگال) دشمنتان نجات داديم، و در طرف راست كوه طور با شما وعده گذارديم، و من و سلوي بر شما نازل كرديم.د"

context = "ي بني اسرائيل ما شما را از (چنگال) دشمنتان نجات داديم، و در طرف راست كوه طور با شما وعده گذارديم، و من و سلوي بر شما نازل كرديم.د"

question = "خداوند در طرف راست کدام کوه با بنی اسرائیل وعده گذارد؟"

# context = """We went on a trip to Europe. We had our breakfast at 7 am in the morning at \
# the nearby coffee shop. Wore a dark blue over coat for our first visit to Louvre Museum \
# to experience history and art."""

# question = "At what time did we had breakfast?"

prompt = f"question: {question} context: {context} </s>"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generated_ids = model.generate(input_ids,
                                    num_beams=7,
                                    num_return_sequences=7,
                                    top_k=50,
#                                     top_p=.2,
                                    temperature=0.95,

                                    max_length=200)
for generated in generated_ids:
    generated_text = tokenizer.decode(generated,skip_special_tokens=True)
print(generated_text)



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


طور طور 


In [19]:
context = """ي بني اسرائيل ما شما را از (چنگال) دشمنتان نجات داديم،  \
و در طرف راست كوه طور با شما وعده گذارديم، و من و سلوي بر شما نازل كرديم.د"""

question = "خداوند در طرف راست کدام کوه با بنی اسرائیل وعده گذارد؟"
print(context)
print(question)


# context = """We went on a trip to Europe. We had our breakfast at 7 am in the morning at \
# the nearby coffee shop. Wore a dark blue over coat for our first visit to Louvre Museum \
# to experience history and art."""

# question = "At what time did we had breakfast?"
# print(context)
# print(question)

ي بني اسرائيل ما شما را از (چنگال) دشمنتان نجات داديم،  و در طرف راست كوه طور با شما وعده گذارديم، و من و سلوي بر شما نازل كرديم.د
خداوند در طرف راست کدام کوه با بنی اسرائیل وعده گذارد؟


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_text =  f"answer: {question} context: {context} </s>"
input_ids = tokenizer(input_text,
                         return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).input_ids.to(device)
# input_ids = encoded_query["input_ids"]
# attention_mask = input_ids["attention_mask"]
attention_mask = tokenizer(input_text,
                         return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).attention_mask.to(device)

generated_answer = model.generate(input_ids, attention_mask = attention_mask,
                                 max_length=64, top_p=0.95, top_k=50)
for each in generated_answer:
  decoded_answer = tokenizer.decode(each, skip_special_tokens=True)
print("Answer: ", decoded_answer)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Answer:  طور 


# evaluation

In [21]:
import json
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open('/content/drive/MyDrive/dev.json', 'r') as f:
  dev_data_list = json.load(f)

predicted_output = []
ground_truth = []

for qa in dev_data_list:
  ground_truth.append(qa['answers'][0]['text'])

  input_text =  f"answer: {qa['question']} context: {qa['context']} </s>"
  input_ids = tokenizer(input_text,
                          return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).input_ids.to(device)

  attention_mask = tokenizer(input_text,
                          return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).attention_mask.to(device)

  generated_answer = model.generate(input_ids, attention_mask = attention_mask,
                                  max_length=64, top_p=0.95, top_k=50)
  for each in generated_answer:
    decoded_answer = tokenizer.decode(each, skip_special_tokens=True)

  predicted_output.append(decoded_answer)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [42]:
g2 = []
p2 = []

for each in ground_truth:
  g2.append(each.strip())

for each in predicted_output:
  p2.append(each.strip())

In [43]:
from sklearn.metrics import f1_score, precision_score, recall_score

# f1 = f1_score(ground_truth, predicted_output, average='micro')
f1 = f1_score(g2, p2, average='micro')

f1

0.1674641148325359

In [40]:
def compute_exact_match_score(pred, gt):
    num_correct = 0

    for p, g in zip(pred, gt):
        if p.strip() == g.strip():
            num_correct += 1

    exact_match_score = num_correct / len(pred)
    return exact_match_score

exact_match_score = compute_exact_match_score(predicted_output, ground_truth)
print("Exact match score: {:.2f}".format(exact_match_score))


Exact match score: 0.17


In [24]:
ground_truth

['روح',
 'موسی',
 'اسرافکاران',
 'تندبادی است که عذاب دردناکی در آن است',
 'شراب طهوری آمیخته با زنجبیل',
 'نوزده نفر',
 'راه\u200cرفتن',
 'فضل خود',
 'تقوای الهی',
 'غیر خدا',
 'رسالتهای الهی',
 'ظالمان',
 'القاب زشت و ناپسند',
 'وحی',
 'صد تازیانه',
 'شدّت و رنجی که به او رسیده',
 'یک',
 'کسانی که به خدا دروغ می\u200cبندند',
 'زراعتی',
 'آنچه انجام می\u200cدادند',
 'غار',
 'نماز',
 'نیکی، این نیست که رویِ خود را به سوی مشرق و مغرب کنید',
 'تورات و انجیل',
 'مریم',
 'قرآن',
 'صابران',
 'قربانگاهی',
 'ابلاغ آشکار',
 'هر کس همتایی برای خدا قرار دهد',
 'سرانجام نیک',
 'الر',
 'کسی که آیات خدا را تکذیب کرده، و از آن روی گردانده است',
 'ابراهیم',
 'کفاره او',
 'حقّی که از سوی پروردگارشان بود',
 'کافران',
 'کسی که شفاعت به کار بدی کند',
 'خطای مردم',
 'هشت',
 'شاهد و آگاه',
 'آیا تو به مردم گفتی که من و مادرم را بعنوان دو معبود غیر از خدا انتخاب کنید',
 'شیاطین',
 'صاحبان اندیشه',
 'عبادت آنها',
 'کسانی که در زمین هستند',
 'پاک و منزّه است کسی که این را مسخّر ما ساخت، وگرنه ما توانایی تسخیر

In [37]:
predicted_output

['روح ',
 'ابراهیم ',
 'انسان ',
 'احقاف ',
 'شرّ آن روز ',
 'نوزده ',
 'چیزی را همتای خدا قرار ',
 'مردان و زنان ',
 'گناهی ',
 'پیامبر ',
 'امری ',
 'خداوند ',
 'گروه دیگر ',
 'وحی شما ',
 'سوره ای ',
 'شدّت و رنجی که به او رسیده ',
 'یک شب ',
 'مردار ',
 'زراعتی ',
 'آیات پروردگارت ',
 'خورشید ',
 'شکر خدا ',
 'کتاب ',
 'تورات و انجیل ',
 'مریم ',
 'روح الامین ',
 'رسولی از خودتان ',
 'چهارپایان ',
 'قلبش ',
 'چهارپایان ',
 'محبّت امور مادی ',
 'الر ',
 'موسی ',
 'ابراهیم ',
 'تورات ',
 'راه خدا ',
 'جبرئیل ',
 'کافران ',
 'آمرزش پروردگارتان ',
 'هفت شب و هشت روز ',
 'تقوای ',
 'بن مریم ',
 'فرزندان فراوانی ',
 'کسانی که دعوت او را اجابت کردند ',
 'خدا ',
 'پیامبرانی که پیش از تو بودند ',
 'نعمت پروردگارتان ',
 'هیچیک از جهانیان ',
 'صالح ',
 'مردگان ',
 'هفت خوشه ',
 'ثمود ',
 'حساب مردم ',
 'دریا و طعام آن ',
 'دلها یی دارند که با آن نمی فهمند ',
 'زیورهای خود گوساله ای ',
 'مریم ',
 'قرآن ',
 'مردم و حج ',
 'راه خدا ',
 'هر حیوان ناخن دار ',
 'دستگيره محكمي ',
 'یهود ',
 'مشرکان 

In [29]:
for a in dev_data_list:
  print(a['question'])

فرشتگان ذکر شده در قرآن چه کسانی هستند؟
کدام پیامبر از سوی فرعونیان تکذیب شد؟
اعمال چه کسانی برایشان زینت داده شده است؟
قوم عاد به چه بلایی گرفتار شدند؟
جامهای بهشتی لبریز از چه چیزی است؟
چند نفر از فرشتگان نگهبان دوزخ هستند؟
لقمان از پسرش خواست در چه چیزی اعتدال را رعایت کند؟
خداوند مومنان تنگدست را از چه چیز بی‌نیاز می‌سازد؟
مومنان باید چه چیزی را در امور خود پیشه کنند؟
اگر قرآن از سوی چه کسی بود، اختلاف فراوانی در آن می‌یافتند؟
پیامبران پیشین چه چیز را بین مردم تبلیع می‌کردند؟
چه کسانی رستگار نخواهند شد؟
در قرآن سفارش شده است که مردم با چه چیزی یکدیگر را یاد نکنند؟
پیامبر تنها به چه وسیله‌ای مردم را انذار می‌کند؟
هر یک از زن و مرد زناکار را باید چند تازیانه بزنند؟
اگر بعد از چه چیز به انسان نعمت چشانده شود غرق غفلت می‌شود؟
خدایی که بنده‌اش را در چند شب، از مسجد الحرام به مسجد الاقصی برد؟
چه کسانی رستگار نخواهند شد؟
خداوند با آب از زمین چه چیز را خارج می‌سازد که رنگهای مختلف دارد؟
‌خداوند در آخرت انسانها را از چه با خبر می‌کند؟
اصحاب کهف در محل وسیعی از چه مکانی قرار داشتند؟
لقمان به

In [39]:
for i in range(len(predicted_output)):
  if predicted_output[i].strip() == ground_truth[i].strip():
    print('hi')

hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
